In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Flatten,Dense,MaxPooling2D,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#data Agumentation

In [2]:
batchsize=8

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,rotation_range=0.2,shear_range=0.2,
                                zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2,validation_split=0.2)
train_data=train_datagen.flow_from_directory(r'C:\Users\sahin\OneDrive\Desktop\drowsiness_detection\mrlEye_dataset\Prepared_Data\train',
                                            target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training')
validation_data=train_datagen.flow_from_directory(r'C:\Users\sahin\OneDrive\Desktop\drowsiness_detection\mrlEye_dataset\Prepared_Data\train',
                                            target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='validation')

Found 58483 images belonging to 2 classes.
Found 14620 images belonging to 2 classes.


In [4]:
test_datagen=ImageDataGenerator(rescale=1./255)
test_data=test_datagen.flow_from_directory(r'C:\Users\sahin\OneDrive\Desktop\drowsiness_detection\mrlEye_dataset\Prepared_Data\test',
                                            target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 6760 images belonging to 2 classes.


In [5]:
bmodel = InceptionV3(include_top=False,weights='imagenet',input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64,activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation='softmax')(hmodel)
model  = Model(inputs=bmodel.input,outputs=hmodel)
for layer in bmodel.layers:
    layer.trainable=False

KeyboardInterrupt: 

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau

In [ ]:
checkpoint=ModelCheckpoint(r'C:\Users\sahin\OneDrive\Desktop\drowsiness_detection\mrlEye_dataset\Models\models.h5',monitor='val_loss',save_best_only=True,verbose=3)

earlystop=EarlyStopping(monitor='val_loss',patience=7,verbose=3,restore_best_weights=True)

learning_rate=ReduceLROnPlateau(monitor='val_loss',patience=3,verbose=3)

callbacks=[checkpoint,earlystop,learning_rate]
 

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                   epochs=5)

In [ ]:
#Model Evaluation

In [ ]:
acc_tr,loss_tr=model.evaluate(train_data)  #tr=training data
print(acc_tr)
print(loss_tr)

In [ ]:
acc_vr,loss_vr=model.evaluate_generator(validation_data) #vr=validation data
print(acc_vr)
print(loss_vr)

In [ ]:
acc_test,loss_test=model.evaluate_generator(test_data)
print(acc_test)
print(loss_test)